# Decoradores e persistência

Este notebook é parte da série "Decoradores e persistência", adaptada de um minicurso ofertado por [Lucas Castro](https://br.linkedin.com/in/lucasmcastro), da [Evolux](https://evolux.net.br).

## Persistência

A execução de programas e scripts geralmente acontece na memória RAM, uma forma de armazenamento volátil que se perde execuções ou cada vez que o computador é reiniciado.

Para que um dado **persista**, é necessário armazená-lo em disco. No entanto, gerenciar arquivos diretamente não é a única solução possível (nem a mais prática). Para isto, geralmente usamos **bancos de dados**.

Existem diferentes tipos de bancos de dados - neste notebook vamos ver apenas um modelo bastante simples, denominado **chave-valor**. Em poucas palavras, um banco chave-valor armazena um valor associado uma chave -- toda indexação para busca, inserção e remoção é feita sobre as chaves.

Como linguagens de programação geralmente trabalham com valores complexos, é necessário usar padrões de **serialização** para converter um objeto complexo em um valor único e depois reconstruí-lo.

## Serialização

Python oferece suporte nativo a diferentes padrões de serialização, cada um com suas vantagens e desvantagens.

### JSON

Um dos padrões mais usados atualmente na web é o **JSON** (*JavaScript Object Notation*), que pode ser usado com os procedimentos da biblioteca ```json```.

* ```dumps``` converte um objeto para o formato JSON e o armazena como uma string.
* ```loads``` lê uma string contendo um objeto no formato JSON e retorna um objeto. 

In [1]:
from json import dumps, loads

In [2]:
test_dict = {"Logradouro": "Campus universitário", "Número": "S/N", "Bairro": "Lagoa Nova"}

In [3]:
string = dumps(test_dict)
print(string)

{"Bairro": "Lagoa Nova", "Logradouro": "Campus universit\u00e1rio", "N\u00famero": "S/N"}


In [4]:
json_dict = loads(string)
test_dict == json_dict

True

* ```load``` e ```dump``` funcionam de forma análoga aos procedimentos anteriores, mas servem para persistência em arquivos.

In [5]:
from json import dump, load

In [6]:
with open("dump.json", "w", encoding="utf-8") as f:
    dump(test_dict, f)

In [7]:
with open("dump.json", encoding="utf-8") as f:
    read_dict = load(f)

In [8]:
test_dict == read_dict

True

In [10]:
loads(dumps([0,1,2,3,4]))

[0, 1, 2, 3, 4]

In [12]:
loads(dumps("Uma string [ contendo ] isso"))

'Uma string [ contendo ] isso'

### Pickle

Uma outra biblioteca padrão do Python para serialização de objetos é a ```pickle```, que serializa objetos para bytes e vice-versa.

Os procedimentos desta biblioteca seguem a mesma interface dos procedimentos da biblioteca ```json```.

* ```dumps``` converte um objeto para um formato compreendido pelo interpretador Python e o armazena como bytes.
* ```loads``` lê uma string contendo um objeto em um formato compreendido pelo interpretador Python e retorna um objeto. 

In [13]:
from pickle import loads, dumps

In [14]:
string = dumps(test_dict)
print(string)

b'\x80\x03}q\x00(X\x06\x00\x00\x00Bairroq\x01X\n\x00\x00\x00Lagoa Novaq\x02X\n\x00\x00\x00Logradouroq\x03X\x15\x00\x00\x00Campus universit\xc3\xa1rioq\x04X\x07\x00\x00\x00N\xc3\xbameroq\x05X\x03\x00\x00\x00S/Nq\x06u.'


In [15]:
pickle_dict = loads(string)
test_dict == pickle_dict

True

* **Obs**: há pessoas que desencorajam o uso da biblioteca ```pickle``` em favor da biblioteca ```json```. Veja este post: https://www.benfrederickson.com/dont-pickle-your-data/

## Persistindo com Redis

Uma vez que você entenda o conceito de serialização, se torna bastante simples utilizar um banco de dados chave valor.

Neste notebook, nós vamos usar o banco [Redis](https://redis.io), que você precisa instalar separadamente.

Existem várias bibliotecas para integrar Python a Redis. Aqui nós usaremos a biblioteca ```redis```, que pode ser instalada via ```pip```.

### Gerenciando a comunicação com o banco

Assim como é necessário um handler (gerenciador) para se comunicar com um arquivo, é necessário ter um handler para se comunicar com um banco de dados. 

A classe StrictRedis permite criar um handler para comunicação com o Redis. Para isto, é necessário informar o servidor (```host```), porta de comunicação (```port```) e o banco que se deseja acessar (```db```):

In [16]:
from redis import StrictRedis

In [17]:
r = StrictRedis(host='localhost', port=6379, db=0)

### Persistindo e recuperando um dado

Um objeto StrictHandler possui os métodos ```set``` e ```get``` para persistir e recuperar um dado, respectivamente.

Ambos exigem que uma chave de identificação do valor seja informada:

In [18]:
r.set('minha_chave', dumps(test_dict))

True

In [19]:
redis_dict = loads(r.get("minha_chave"))
test_dict == redis_dict

True

Note que no exemplo acima o objeto foi persistido usando os procedimento ```dumps``` e ```loads``` da biblioteca ```pickle```. 

Também seria possível persistir o objeto usando os procedimentos ```dumps``` e ```loads``` da biblioteca ```json```. 

No entanto, como o método ```get``` retorna um objeto ```bytes```, é necessário convertê-lo para ```str``` antes de usar o procedimento ```loads```. 

Isto pode ser feito com o método ```bytes.decode()```.

In [20]:
from json import dumps, loads

In [21]:
r.set('minha_chave', dumps(test_dict))

True

In [22]:
redis_dict = loads(r.get("minha_chave").decode())
test_dict == redis_dict

True